In [1]:
import pandas as pd
import altair as alt
from typing import List
from visualize import timeline

alt.data_transformers.disable_max_rows()

ModuleNotFoundError: No module named 'visualize'

In [3]:
def most_significant(l, i) -> List[str]:
    df = pd.concat(l)
    ms = (
        df.groupby("element")["KL"]
        .apply(lambda c: c.abs().sum())
        .sort_values(ascending=False)
        .head(i)
        .index.to_list()
    )
    ms = [e for e in ms if e not in ["laden", "osama", "marilyn","michael jackson", "hussein"]]
    return df[df["element"].isin(ms)].reset_index(drop=True), ms

In [12]:
def element_kv():
    k = ["שם יישוב", "סמל יישוב"]
    v = ["name", "element"]
    kv = pd.read_excel("data/elections/bycode2021.xlsx")[k].rename(
        columns=dict(zip(k, v))
    )
    return pd.concat(
        [kv, pd.DataFrame(["מעטפות חיצוניות", 99999], index=v).T], ignore_index=True
    )

ekv = element_kv()
ekv

,name,element
0,אבו ג'ווייעד (שבט),967
1,אבו גוש,472
2,אבו סנאן,473
3,אבו סריחאן (שבט),935
4,אבו עבדון (שבט),958
...,...,...
1479,תקוע,3563
1480,תראבין א-צאנע (שבט),970
1481,תרבין א-צאנע (יישוב)*,1346
1482,תרום,778


### vis 1
Frequencies of top 15 words in the dvrs

In [15]:
corpus = "elections"
dvr = (
    pd.read_csv(f"results/{corpus}/dvr.csv")
    .sort_values("global_weight", ascending=False)["element"]
    .head(15)
)
squeezed_freq = pd.read_csv(
    f"data/{corpus}/np_freq/0.csv"
)
pdf = squeezed_freq[squeezed_freq["element"].isin(dvr)].reset_index(drop=True)
pdf["element"] = pdf["element"].replace(ekv.set_index("element").to_dict()["name"])
display(
    timeline(
        pdf,
        x="document",
        y="frequency_in_document",
        subcorpus=corpus,
        stack="center",
        order=dvr.to_list(),
        name=f"vis_1_{corpus}_timeline",
    )
)


/home/alexzabbey/.virtualenvs/lpa/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

### vis 2
Distances from dvr for most significant words

In [5]:
for subcorpus in ("mainstream", "conspiracy"):
    df = pd.read_csv(f"results/{subcorpus}/max_distances.csv")
    ms = df.columns.to_list()
    df = df.assign(
        **{
            "category": sorted([
                p.split("/")[-1][5:-4] for p in glob(f"results/{subcorpus}/sigs/*.csv")
            ])
        }
    )
    msdf = pd.melt(df[ms + ["category"]], id_vars="category", var_name="element", value_name="KL")
    display(
        timeline(
            msdf,
            x="category",
            y="KL",
            subcorpus=subcorpus,
            stack=None,
            order=ms,
            name=f"vis_2_{subcorpus}_timeline",
        )
    )


alt.Chart(...)

alt.Chart(...)

### vis 3
Frequencies of most significant words distance wise

In [6]:
for subcorpus in ("mainstream", "conspiracy"):
    df = pd.read_csv(f"results/{subcorpus}/max_distances.csv")
    ms = df.columns.to_list()
    squeezed_freq = pd.read_csv(
        f"results/{subcorpus}/squeezed_freq.csv", parse_dates=["date"]
    )
    pdf = squeezed_freq[squeezed_freq["element"].isin(ms)].reset_index(drop=True)
    pdf["xdate"] = pdf["date"].astype(str).str[:7]
    display(
        timeline(
            pdf,
            x="xdate",
            y="global_weight",
            subcorpus=subcorpus,
            stack="center",
            order=ms,
            name=f"vis_3_{subcorpus}_timeline",
        )
    )


alt.Chart(...)

alt.Chart(...)